In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

# Load the training and test datasets
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')

# Display the first few rows of the training data
train_df.head()

In [ ]:
# Fill missing values in 'Age' with the median
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)

# Fill missing values in 'Fare' with the median
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

# Fill missing values in 'Embarked' with the mode (most frequent value)
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)

# Drop the 'Cabin' column because it has too many missing values
train_df.drop(columns=['Cabin'], inplace=True)
test_df.drop(columns=['Cabin'], inplace=True)

In [ ]:
# Encode 'Sex' (0 for male, 1 for female)
train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1})
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1})

# Encode 'Embarked' (0 for S, 1 for C, 2 for Q)
train_df['Embarked'] = train_df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
test_df['Embarked'] = test_df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

In [ ]:
# Create 'FamilySize' feature
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

# Extract 'Title' from names
train_df['Title'] = train_df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
test_df['Title'] = test_df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())

# Replace rare titles with 'Other'
rare_titles = ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona']
train_df['Title'] = train_df['Title'].replace(rare_titles, 'Other')
test_df['Title'] = test_df['Title'].replace(rare_titles, 'Other')

# Map titles to numerical values
title_mapping = {'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Other': 4}
train_df['Title'] = train_df['Title'].map(title_mapping)
test_df['Title'] = test_df['Title'].map(title_mapping)

In [ ]:
# Select features for clustering
features_for_clustering = train_df[['Pclass', 'Age', 'Fare', 'FamilySize']]

# Now check for missing values
print(features_for_clustering.isnull().sum())

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Standardize the features for clustering
scaler = StandardScaler()
features_for_clustering_scaled = scaler.fit_transform(train_df[['Pclass', 'Age', 'Fare', 'FamilySize']])

# Apply KMeans clustering
kmeans = KMeans(n_clusters=5, random_state=42)
train_df['Cluster'] = kmeans.fit_predict(features_for_clustering_scaled)

# Apply the same scaling and clustering to the test set
test_features_scaled = scaler.transform(test_df[['Pclass', 'Age', 'Fare', 'FamilySize']])
test_df['Cluster'] = kmeans.predict(test_features_scaled)

# Visualize the clusters (optional, for understanding the distribution)
plt.scatter(train_df['Fare'], train_df['Age'], c=train_df['Cluster'], cmap='viridis')
plt.xlabel('Fare')
plt.ylabel('Age')
plt.title('Passenger Clusters based on Fare and Age')
plt.show()


In [ ]:
# Fill missing values in 'Title' with the most common value in both train and test sets
X_train['Title'] = X_train['Title'].fillna(X_train['Title'].mode()[0])
X_test['Title'] = X_test['Title'].fillna(X_test['Title'].mode()[0])

# Check again for missing values (should be 0 now)
print(X_train.isnull().sum())
print(X_test.isnull().sum())

# Now proceed with training the model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=300, max_depth=10, min_samples_split=5, random_state=42)

# Evaluate the model using cross-validation
scores = cross_val_score(rf_model, X_train, y_train, cv=5)

# Print cross-validation scores and the mean accuracy
print(f'Cross-validation scores: {scores}')
print(f'Mean accuracy: {scores.mean() * 100:.2f}%')


In [ ]:
# Check for missing values in X_train
print(X_train.isnull().sum())


In [ ]:
# Fill missing values in 'Title' with the most common value (or with 'Other')
X_train['Title'].fillna(X_train['Title'].mode()[0], inplace=True)
X_test['Title'].fillna(X_test['Title'].mode()[0], inplace=True)

# Check again for missing values
print(X_train.isnull().sum())


In [ ]:
# Create X_test by selecting relevant features
X_test = test_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'FamilySize', 'Title', 'Cluster']]

# Fill missing values in 'Title' with the most common value (or 'Other')
X_test['Title'] = X_test['Title'].fillna(X_test['Title'].mode()[0])

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the hyperparameters grid to search
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the best score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_ * 100:.2f}%")


In [ ]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score

# Initialize XGBoost model
xgb_model = xgb.XGBClassifier(n_estimators=300, max_depth=10, learning_rate=0.05, random_state=42)

# Cross-validation for XGBoost
scores = cross_val_score(xgb_model, X_train, y_train, cv=5)

# Print cross-validation results
print(f'XGBoost cross-validation scores: {scores}')
print(f'Mean accuracy: {scores.mean() * 100:.2f}%')


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# Initialize models
logreg = LogisticRegression()
rf = RandomForestClassifier(n_estimators=300, max_depth=10, random_state=42)
xgb_model = xgb.XGBClassifier(n_estimators=300, max_depth=10, learning_rate=0.05, random_state=42)

# Combine them using VotingClassifier
ensemble_model = VotingClassifier(estimators=[
    ('lr', logreg),
    ('rf', rf),
    ('xgb', xgb_model)
], voting='soft')

# Cross-validation for the ensemble model
scores = cross_val_score(ensemble_model, X_train, y_train, cv=5)

print(f'Ensemble model cross-validation scores: {scores}')
print(f'Mean accuracy: {scores.mean() * 100:.2f}%')
 

In [ ]:
# Create age bins
train_df['AgeBin'] = pd.cut(train_df['Age'], bins=[0, 12, 20, 40, 60, 80], labels=[0, 1, 2, 3, 4])
test_df['AgeBin'] = pd.cut(test_df['Age'], bins=[0, 12, 20, 40, 60, 80], labels=[0, 1, 2, 3, 4])


In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Define base models
estimators = [
    ('rf', RandomForestClassifier(n_estimators=300, max_depth=10, random_state=42)),
    ('xgb', xgb.XGBClassifier(n_estimators=300, max_depth=10, learning_rate=0.05, random_state=42))
]

# Define Stacking model with Logistic Regression as meta-model
stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

# Cross-validation for the stacking model
scores = cross_val_score(stacking_model, X_train, y_train, cv=5)

print(f'Stacking model cross-validation scores: {scores}')
print(f'Mean accuracy: {scores.mean() * 100:.2f}%')


In [ ]:
# Create a new feature combining Pclass and Fare
X_train['Pclass_Fare'] = X_train['Pclass'] * X_train['Fare']
X_test['Pclass_Fare'] = X_test['Pclass'] * X_test['Fare']


In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
scores = cross_val_score(rf_model, X_train, y_train, cv=skf)

print(f'StratifiedKFold cross-validation scores: {scores}')
print(f'Mean accuracy: {scores.mean() * 100:.2f}%')


In [ ]:
# Re-train the best Random Forest model from GridSearchCV
best_rf_model = RandomForestClassifier(
    n_estimators=200, max_depth=20, min_samples_leaf=2, min_samples_split=10, random_state=42
)

# Train the model on the entire training set
best_rf_model.fit(X_train, y_train)


In [ ]:
# Make predictions on the test set
test_predictions = best_rf_model.predict(X_test)

# Create a submission DataFrame
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': test_predictions
})


In [ ]:
# Save the submission to a CSV file
submission.to_csv('submission.csv', index=False)

print("Submission file created successfully.")


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

# Paramètres possibles pour XGBoost
param_dist = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 5, 10, 15],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 5],
    'reg_alpha': [0, 0.01, 0.1, 1],
    'reg_lambda': [0.01, 0.1, 1]
}

# Initialiser XGBoost
xgb_model = xgb.XGBClassifier(random_state=42)

# Randomized Search avec 5-fold cross-validation
random_search = RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=20, cv=5, verbose=2, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres et la meilleure précision
print(f"Best hyperparameters: {random_search.best_params_}")
print(f"Best cross-validation accuracy: {random_search.best_score_ * 100:.2f}%")


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score

# Initialiser LightGBM
lgb_model = lgb.LGBMClassifier(n_estimators=300, max_depth=10, learning_rate=0.05, random_state=42)

# Validation croisée sur LightGBM
scores = cross_val_score(lgb_model, X_train, y_train, cv=5)

print(f'LightGBM cross-validation scores: {scores}')
print(f'Mean accuracy: {scores.mean() * 100:.2f}%')


In [ ]:
# Binning Age and Fare
train_df['FareBin'] = pd.cut(train_df['Fare'], bins=[0, 8, 15, 31, 1000], labels=[0, 1, 2, 3])
test_df['FareBin'] = pd.cut(test_df['Fare'], bins=[0, 8, 15, 31, 1000], labels=[0, 1, 2, 3])

train_df['AgeBin'] = pd.cut(train_df['Age'], bins=[0, 12, 20, 40, 60, 80], labels=[0, 1, 2, 3, 4])
test_df['AgeBin'] = pd.cut(test_df['Age'], bins=[0, 12, 20, 40, 60, 80], labels=[0, 1, 2, 3, 4])


In [ ]:
# Interaction entre Pclass et Fare
train_df['Pclass_Fare'] = train_df['Pclass'] * train_df['Fare']
test_df['Pclass_Fare'] = test_df['Pclass'] * test_df['Fare']


In [ ]:
# Séparation des données en fonction du sexe
train_femmes = train_df[train_df['Sex'] == 1]  # Femmes
train_hommes = train_df[train_df['Sex'] == 0]  # Hommes

test_femmes = test_df[test_df['Sex'] == 1]
test_hommes = test_df[test_df['Sex'] == 0]


In [ ]:
# Vérifier les valeurs manquantes dans X_train_femmes
print(X_train_femmes.isnull().sum())


In [ ]:
# Réentraîner le modèle après avoir traité les NaN
rf_model_femmes.fit(X_train_femmes, y_train_femmes)

# Prédictions pour les femmes
predictions_femmes = rf_model_femmes.predict(X_test_femmes)


In [ ]:
# Séparation des données en fonction du sexe
train_hommes = train_df[train_df['Sex'] == 0]  # Hommes
test_hommes = test_df[test_df['Sex'] == 0]

# Sélection des features pour les hommes
X_train_hommes = train_hommes[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'FamilySize', 'Title', 'FareBin', 'AgeBin', 'Pclass_Fare']]
y_train_hommes = train_hommes['Survived']

X_test_hommes = test_hommes[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'FamilySize', 'Title', 'FareBin', 'AgeBin', 'Pclass_Fare']]


In [ ]:
# Remplir les valeurs manquantes dans 'Age' et 'Fare'
X_train_hommes['Age'].fillna(X_train_hommes['Age'].median(), inplace=True)
X_train_hommes['Fare'].fillna(X_train_hommes['Fare'].median(), inplace=True)

# Remplir les NaN dans 'Title'
X_train_hommes['Title'].fillna(X_train_hommes['Title'].mode()[0], inplace=True)

# Appliquer les mêmes transformations au jeu de test des hommes
X_test_hommes['Age'].fillna(X_test_hommes['Age'].median(), inplace=True)
X_test_hommes['Fare'].fillna(X_test_hommes['Fare'].median(), inplace=True)
X_test_hommes['Title'].fillna(X_test_hommes['Title'].mode()[0], inplace=True)


In [ ]:
# Vérifier les valeurs manquantes dans X_train_hommes
print(X_train_hommes.isnull().sum())


In [ ]:
# Remplir les valeurs manquantes dans 'FareBin' en utilisant la valeur la plus fréquente (mode)
X_train_hommes['FareBin'].fillna(X_train_hommes['FareBin'].mode()[0], inplace=True)
X_test_hommes['FareBin'].fillna(X_test_hommes['FareBin'].mode()[0], inplace=True)


In [ ]:
# Entraîner le modèle pour les hommes
rf_model_hommes = RandomForestClassifier(n_estimators=300, max_depth=10, random_state=42)
rf_model_hommes.fit(X_train_hommes, y_train_hommes)

# Prédictions pour les hommes
predictions_hommes = rf_model_hommes.predict(X_test_hommes)
